In [9]:
import sqlite3
import pandas as pd
import time

# initiate time variable to determine how long it would take to run each block of script
st = time.time()

# connect to sqlite database and retrieve plain data - financial ratio and stock price
# close connection when it is done
conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

# conclude time variable for 1st block
# initiate time variable for 2nd block
print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

# retrieve all financial ratio of eps only, can change to apply for bvs and so on
#ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'BVS']
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']

# clean the raw financial ratio data to ensure its credibility 
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('^-$', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

# conclude time variable for 2nd block
# initiate time variable for 3rd block
print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

# clean the raw stock price data to ensure its credibility 
# extract data (year/quarter) where necessary
ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

# conclude time variable for 3rd block
# initiate time variable for 4th block
print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

# manipulate price table to get price of current year/quarter lined up with its respective next year/quarter
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).nth(0)
ds_price['NEXT-PRICE-Q'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

# conclude time variable for 4th block
# initiate time variable for 5th block
print("4--- %s seconds ---" % (time.time() - st))
st = time.time()

# merge price table into ratio table, then calculate ep/return,  all rows with null ep will be taken out
# qcut(10) the result to get decile information, separated for each year
ds_ratio = ds_ratio.merge(ds_price, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='right')
ds_ratio['EP'] = ds_ratio['STOCK_DATA'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio = ds_ratio.loc[ds_ratio['EP'].notnull()]
ds_ratio['RETURN'] = ds_ratio['NEXT-PRICE-Y'].div(ds_ratio['NEXT-PRICE-Q'])
ds_ratio['DECILE'] = (ds_ratio.groupby('STOCK_YEAR')['EP']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))

# manipulate ratio table to calculate average return
# create s1 as table to hold final result of the strategy
s1 = ds_ratio.groupby(['STOCK_YEAR','DECILE'], as_index=False).mean()

# conclude time variable for 5th block
# initiate time variable for next block when applicable
print("5--- %s seconds ---" % (time.time() - st))
st = time.time()

# pivot s1 table to show respective columns, rows and return data
s1.pivot(index='DECILE', columns='STOCK_YEAR', values='RETURN')

1--- 2.772279739379883 seconds ---
2--- 0.368375301361084 seconds ---
3--- 5.79455828666687 seconds ---


STOCK_CLOSE STOCK_DATE  STOCK_YEAR  NEXT-PRICE-Q
STOCK_CODE STOCK_QUARTER                                                  
AAA        20164              16141.0 2016-12-30        2016       16900.0
           20171              16900.0 2017-03-31        2017       23833.0
           20172              23833.0 2017-06-30        2017       25230.0
           20173              25230.0 2017-09-29        2017       24383.0
           20174              24383.0 2017-12-29        2017       19103.0
           20181              19103.0 2018-03-30        2018       18900.0
           20182              18900.0 2018-06-29        2018       17500.0
           20183              17500.0 2018-09-24        2018           NaN
AAM        20113              13935.0 2011-09-30        2011       13621.0
           20114              13621.0 2011-12-30        2011       15209.0
           20121              15209.0 2012-03-30        2012       16928.0
           20122              16928.0 2012-06-29        2012       16033.0
           20123              16033.0 2012-09-28        2012       16498.0
           20124              16498.0 2012-12-28        2012       15966.0
           20131              15966.0 2013-03-29        2013       14636.0
           20132              14636.0 2013-06-28        2013       14088.0
           20133              14088.0 2013-09-30        2013       11114.0
           20134              11114.0 2013-12-31        2013       13274.0
           20141              13274.0 2014-03-31        2014       12105.0
           20142              12105.0 2014-06-30        2014       12856.0
           20143              12856.0 2014-09-30        2014       12022.0
           20144              12022.0 2014-12-31        2014       11527.0
           20151              11527.0 2015-03-31        2015       11527.0
           20152              11527.0 2015-06-30        2015        9366.0
           20153               9366.0 2015-09-30        2015        9923.0
           20154               9923.0 2015-12-31        2015       10400.0
           20161              10400.0 2016-03-31        2016       10400.0
           20162              10400.0 2016-06-30        2016       10209.0
           20163              10209.0 2016-09-30        2016        9083.0
           20164               9083.0 2016-12-30        2016        9732.0
...                               ...        ...         ...           ...
XPH        20151              15300.0 2015-03-31        2015        9191.0
           20152               9191.0 2015-06-30        2015        8036.0
           20153               8036.0 2015-09-30        2015        8451.0
           20154               8451.0 2015-12-31        2015       11057.0
           20161              11057.0 2016-03-31        2016        8267.0
           20162               8267.0 2016-06-30        2016        7000.0
           20163               7000.0 2016-09-30        2016        6200.0
           20164               6200.0 2016-12-30        2016        7500.0
           20171               7500.0 2017-03-31        2017        6295.0
           20172               6295.0 2017-06-30        2017        5700.0
           20173               5700.0 2017-09-29        2017        6800.0
           20174               6800.0 2017-12-29        2017        7000.0
           20181               7000.0 2018-03-30        2018        7300.0
           20182               7300.0 2018-06-29        2018        4900.0
           20183               4900.0 2018-09-24        2018           NaN
YBC        20142               5200.0 2014-06-30        2014        4400.0
           20143               4400.0 2014-09-30        2014        2500.0
           20144               2500.0 2014-12-31        2014        3200.0
           20151               3200.0 2015-03-31        2015        5600.0
           20152               5600.0 2015-06-30        2015        5000.0
           20153               5000.0 2015-09-30   